In [1]:
%matplotlib inline
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dqnroute.utils import *

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


ImportError: No module named 'networkx'

In [52]:
def mk_num_list(s, n):
    return list(map(lambda k: s+str(k), range(0, n)))

meta_cols = ['time', 'pkg_id', 'cur_node']

def get_target_cols(n):
    return mk_num_list('predict_', 10)

def get_dst_cols(n):
    return mk_num_list('dst_', n)

def get_addr_cols(n):
    return mk_num_list('addr_', n)

def get_neighbors_cols(n):
    return mk_num_list('neighbors_', n)

def get_feature_cols(n):
    return get_dst_cols(n) + get_addr_cols(n) + get_neighbors_cols(n)

def get_amatrix_cols(n):
    res = []
    for m in range(0, n):
        s = 'amatrix_'+str(m)+'_'
        res += mk_num_list(s, n)
    return res
        
def data_cols(n):
    return meta_cols + get_feature_cols(n) + get_amatrix_cols(n) + get_target_cols(n)


In [43]:
data = pd.read_csv('logs/data4.csv', names=data_cols(10))

In [44]:
data.head()

,time,pkg_id,cur_node,dst_0,dst_1,dst_2,dst_3,dst_4,dst_5,dst_6,...,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,15.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,31.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,25.0,2.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1000000.0,-1000000.0,-1000000.0,-10.0,-1000000.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
3,41.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0
4,45.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-10.0,-1000000.0


In [81]:
target_cols = get_target_cols(10)
feature_cols = data_cols(10)
targets = data[target_cols]
features = data[feature_cols]

In [82]:
features.head()

,time,pkg_id,cur_node,dst_0,dst_1,dst_2,dst_3,dst_4,dst_5,dst_6,...,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
0,15.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-10.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-30.0,-1000000.0
1,31.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1000000.0,-20.0,-20.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
2,25.0,2.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1000000.0,-1000000.0,-1000000.0,-10.0,-1000000.0,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0
3,41.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0,-20.0,-1000000.0,-1000000.0
4,45.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-30.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-1000000.0,-30.0,-1000000.0,-10.0,-1000000.0


In [83]:
input_dim = features.shape[1]
output_dim = targets.shape[1]

In [84]:
model = Sequential()
model.add(Dense(output_dim=64, activation='relu', input_dim=input_dim))
model.add(Dense(output_dim=64, activation='relu'))
model.add(Dense(output_dim=output_dim, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

In [85]:
X_train, X_test, y_train, y_test = train_test_split(features.values, targets.values, test_size=0.2, random_state=42)

In [87]:
model.fit(X_train, y_train, nb_epoch=1000)

Epoch 1/1000
2683/2683 [==============================] - 0s - loss: 71874400.8781     
Epoch 2/1000
2683/2683 [==============================] - 0s - loss: 68838559.0071     
Epoch 3/1000
2683/2683 [==============================] - 0s - loss: 70578955.5035     

In [59]:
model.evaluate(X_test, y_test)

 32/671 [>.............................] - ETA: 0s

12223.168683343423

In [65]:
model.predict(X_train[0].reshape((1, 30)))

array([[ -9.99989500e+05,   4.69137802e+01,   1.12038445e+02,
         -9.99917000e+05,  -9.99879938e+05,  -9.99869688e+05,
         -9.99970062e+05,  -9.53002701e+01,  -9.99818938e+05,
         -9.99989938e+05]], dtype=float32)

In [66]:
y_train[0]

array([ -1.00000000e+06,  -2.00000000e+01,  -2.00000000e+01,
        -1.00000000e+06,  -1.00000000e+06,  -1.00000000e+06,
        -1.00000000e+06,  -4.00000000e+01,  -1.00000000e+06,
        -1.00000000e+06])